# PydanticAI fundamentals 

In [2]:
from pydantic_ai import Agent

agent = Agent(model="google-gla:gemini-2.5-flash")

result = await agent.run(" Give me a IT employee working in Sweden, no yapping just the result ")

result

AgentRunResult(output="**Name:** Anders Nilsson\n**Age:** 34\n**Role:** Senior DevOps Engineer\n**Company:** Klarna (Fintech)\n**Location:** Stockholm, Sweden\n\nAnders works in a cross-functional team, focusing on automating deployment pipelines, managing cloud infrastructure (AWS), and ensuring system reliability for Klarna's payment solutions. He typically cycles to their city-center office, enjoys a communal fika break with colleagues, and practices flexible working hours. In his free time, he's often out with his family, hiking in nearby nature reserves or enjoying the city's cultural offerings.")

In [4]:
print(result.output)

**Name:** Anders Nilsson
**Age:** 34
**Role:** Senior DevOps Engineer
**Company:** Klarna (Fintech)
**Location:** Stockholm, Sweden

Anders works in a cross-functional team, focusing on automating deployment pipelines, managing cloud infrastructure (AWS), and ensuring system reliability for Klarna's payment solutions. He typically cycles to their city-center office, enjoys a communal fika break with colleagues, and practices flexible working hours. In his free time, he's often out with his family, hiking in nearby nature reserves or enjoying the city's cultural offerings.


## Use pydantic to stucture output

In [11]:
from pydantic import BaseModel, Field
class Employee(BaseModel):
    name: str = Field(description="The full name of the employee")
    age: int = Field(gt = 17,lt= 67 ,description="The age of the employee")
    salary: int = Field(gt = 30_000, lt= 100_000, description="The monthly salary of the employee in SEK, the more sineor the higher salary")
    role: str = Field(description="The role of the employee in the company, allways include the level of seniority e.g. Junior, Mid, Senior")
    description: str = Field(description="A short description of the employee, max 20 words")

result = await agent.run(" Give me a data engineer working in Sweden, no yapping just the result", output_type=Employee)
result 


AgentRunResult(output=Employee(name='Erik Johansson', age=30, salary=55000, role='Mid-level Data Engineer', description='Skilled in building and maintaining data pipelines and infrastructure, ensuring data quality and accessibility.'))

In [13]:
employee = result.output
employee

Employee(name='Erik Johansson', age=30, salary=55000, role='Mid-level Data Engineer', description='Skilled in building and maintaining data pipelines and infrastructure, ensuring data quality and accessibility.')

In [14]:
employee.description, employee.name

('Skilled in building and maintaining data pipelines and infrastructure, ensuring data quality and accessibility.',
 'Erik Johansson')

In [15]:
isinstance(employee, BaseModel)

True

In [16]:
employee.model_dump()

{'name': 'Erik Johansson',
 'age': 30,
 'salary': 55000,
 'role': 'Mid-level Data Engineer',
 'description': 'Skilled in building and maintaining data pipelines and infrastructure, ensuring data quality and accessibility.'}

In [17]:
print(employee.model_dump_json(indent=3))

{
   "name": "Erik Johansson",
   "age": 30,
   "salary": 55000,
   "role": "Mid-level Data Engineer",
   "description": "Skilled in building and maintaining data pipelines and infrastructure, ensuring data quality and accessibility."
}


## Simulate 10 employee

In [18]:
result = await agent.run("give me 10 different IT employees working in Sweden, no yapping just the result", output_type=list[Employee])

result

AgentRunResult(output=[Employee(name='Astrid Lindgren', age=32, salary=55000, role='Mid-level Software Engineer', description='Experienced in Python and JavaScript development, focusing on backend systems and API integrations.'), Employee(name='Björn Borg', age=45, salary=85000, role='Senior DevOps Engineer', description='Specializes in cloud infrastructure, CI/CD pipelines, and ensuring system reliability and scalability.'), Employee(name='Carl Larsson', age=27, salary=40000, role='Junior Data Analyst', description='Eager to learn, proficient in SQL and basic data visualization tools, supporting business intelligence.'), Employee(name='Doris Dahl', age=38, salary=68000, role='Lead UI/UX Designer', description='Designs intuitive user experiences and interfaces, leading a small team of designers.'), Employee(name='Erik Johansson', age=50, salary=95000, role='Principal Solutions Architect', description='Provides strategic technical direction and designs complex enterprise-level software 

In [20]:
for employee in result.output:
    print(f"{employee.name} is {employee.age} years old and works as a {employee.role} earning {employee.salary} SEK per month.")

Astrid Lindgren is 32 years old and works as a Mid-level Software Engineer earning 55000 SEK per month.
Björn Borg is 45 years old and works as a Senior DevOps Engineer earning 85000 SEK per month.
Carl Larsson is 27 years old and works as a Junior Data Analyst earning 40000 SEK per month.
Doris Dahl is 38 years old and works as a Lead UI/UX Designer earning 68000 SEK per month.
Erik Johansson is 50 years old and works as a Principal Solutions Architect earning 95000 SEK per month.
Fanny Åberg is 29 years old and works as a Mid-level QA Engineer earning 48000 SEK per month.
Gustav Vasa is 42 years old and works as a Senior Cybersecurity Analyst earning 75000 SEK per month.
Helena Bergström is 35 years old and works as a Mid-level Data Scientist earning 60000 SEK per month.
Ingrid Bergman is 55 years old and works as a IT Project Manager earning 98000 SEK per month.
Johan Karlsson is 30 years old and works as a Mid-level Full-stack Developer earning 52000 SEK per month.


## More complex model

In [26]:
from typing import Literal

class Department(BaseModel):
    name: Literal["IT", "HR", "Marketing", "Sales", "Finance"] = Field(description="The name of the department")
    description: str = Field(description="A short description of the department")
    employees: list[Employee]

class Company(BaseModel):
    name: str = Field(description="The name of the company")
    departments: list[Department]
    description: str = Field(description="A short description of the company")


result = await agent.run(("give me 5 employees in ench department, give me the departments IT, HR, Marketing, Sales and Finance, no yapping just the result",
                         "company should be called CoolCompany and be based in Sweden and sell ice cream"),
                          output_type=Company)

result

AgentRunResult(output=Company(name='CoolCompany', departments=[Department(name='IT', description='Manages all technology and infrastructure within the company.', employees=[Employee(name='Alice Wonderland', age=45, salary=65000, role='Senior IT Architect', description='Oversees system architecture and technical strategy.'), Employee(name='Bob The Builder', age=32, salary=48000, role='Mid-level Software Developer', description='Develops and maintains company software solutions.'), Employee(name='Charlie Chaplin', age=25, salary=32000, role='Junior IT Support Technician', description='Provides first-line technical support to employees.'), Employee(name='Diana Prince', age=40, salary=62000, role='Senior Cyber Security Analyst', description='Protects company data and systems from threats.'), Employee(name='Eve Adams', age=35, salary=50000, role='Mid-level Database Administrator', description='Manages and maintains company databases.')]), Department(name='HR', description='Handles all human

In [28]:
company = result.output

company.model_dump()


{'name': 'CoolCompany',
 'departments': [{'name': 'IT',
   'description': 'Manages all technology and infrastructure within the company.',
   'employees': [{'name': 'Alice Wonderland',
     'age': 45,
     'salary': 65000,
     'role': 'Senior IT Architect',
     'description': 'Oversees system architecture and technical strategy.'},
    {'name': 'Bob The Builder',
     'age': 32,
     'salary': 48000,
     'role': 'Mid-level Software Developer',
     'description': 'Develops and maintains company software solutions.'},
    {'name': 'Charlie Chaplin',
     'age': 25,
     'salary': 32000,
     'role': 'Junior IT Support Technician',
     'description': 'Provides first-line technical support to employees.'},
    {'name': 'Diana Prince',
     'age': 40,
     'salary': 62000,
     'role': 'Senior Cyber Security Analyst',
     'description': 'Protects company data and systems from threats.'},
    {'name': 'Eve Adams',
     'age': 35,
     'salary': 50000,
     'role': 'Mid-level Database A

In [34]:
departments = [department.name for department in company.departments]
departments

['IT', 'HR', 'Marketing', 'Sales', 'Finance']